In [38]:
import numpy as np

5.1 Написать функцию формирования порождающей матрицы кода Рида-
Маллера (r,m) в каноническом виде для произвольно заданных r и m.

In [39]:
def bin_generator(m):
  a = np.zeros((2**m, m), dtype=int)
  for i in range(2**m):
    temp = [int(k) for k in bin(i)[2:].zfill(m)]
    a[i] = np.array(temp[-1:-(m+1):-1])
  return a


def f(a, I):
  m = a.shape[1]
  v = np.zeros(2**m, dtype=int)
  for i in range(2**m):
    temp = 1
    for j in I:
      temp *= a[i][j] ^ 1
    v[i] = temp
  return v


def number_of_combinations(i, m):
  return np.math.factorial(m) / np.math.factorial(m-i) / np.math.factorial(i)


def get_size(r, m):
  res = 0
  for i in range(r+1):
    res += number_of_combinations(i, m)
  return res


def I_generator(r, m):
  if r < 0:
    return []
  size1 = number_of_combinations(r, m)
  I_list = []
  a = [i for i in range(m-1, -1, -1)]
  i = 0
  c = 0
  while c < size1:
    temp = [int(k) for k in bin(i)[2:].zfill(m)]
    if sum(temp) == r:
      w = []
      for k in range(m):
        if temp[k]:
          w.append(a[k])
      c += 1
      I_list.append(w)
    i += 1
    I_list.sort(reverse=True)
    b = list(map(lambda x: list(reversed(x)), I_list))

  return list(I_generator(r-1, m)) + b


def G_generator(r, m):
  G = np.zeros((int(get_size(r,m)), 2**m), dtype=int)
  I = I_generator(r, m)
  a = bin_generator(m)
  for i in range(int(get_size(r,m))):
    G[i] = f(a, I[i])
  return G

5.2. Реализовать алгоритм мажоритарного декодирования для кода Рида-
Маллера.

In [40]:
def f_t(a, Jc, t):
  m = a.shape[1]
  v = np.zeros(2**m, dtype=int)
  for i in range(2**m):
    temp = 1
    for j in Jc:
      temp *= (a[i][j] ^ t[j]) ^ 1
    v[i] = temp
  return v


def RM_decoder(w, r, m):
  u = []
  while r >= 0:
    J_list = list(filter(lambda x: len(x) == r, I_generator(r, m)))
    #print("J_list", J_list)
    base = [i for i in range(m)]
    Jc_list = list(map(lambda x: [item for item in base if item not in x], J_list))
    #print("Jc_list", Jc_list)

    a = bin_generator(m)
    H_list = []
    for i in range(len(J_list)):
      v = f(a, J_list[i])
      H_list.append([a[i] for i in range(len(v)) if v[i] == 1])

    m_list = []

    #print("H_list", H_list)
    for i in range(len(H_list)):
      count1 = 0
      count0 = 0
      for k in H_list[i]:
        if np.sum(w * np.array(f_t(a, Jc_list[i], k))) % 2:
          count1 += 1
        else:
          count0 += 1
      if count1 == count0:
        #print("count1 == count0", count1)
        print("Не можем исправить")
        return
      m_list.append(1 if count1 > count0 else 0)

    for i in range(len(m_list)):
      if m_list[i]:
        w ^= f(a, J_list[i])
    #print("m_list", m_list)

    #print("w", w)
    r -= 1
    u = m_list + u
  return np.array(u)

4.3. Провести экспериментальную проверку алгоритма декодирования для
кода Рида-Маллера (2,4).

In [43]:
r = 2
m = 4
u = np.array([0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0])
G = G_generator(r, m)
w = u @ G

err1 = np.array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
w_problem = w ^ err1
u_new = RM_decoder(w_problem, r, m)
print(u)
print(u_new, "\n")


err2 = np.array([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
w_problem = w ^ err2
u_new = RM_decoder(w_problem, r, m)
print(u)
print(u_new)

[0 1 0 0 0 0 0 0 0 1 0]
[0 1 0 0 0 0 0 0 0 1 0] 

Не можем исправить
[0 1 0 0 0 0 0 0 0 1 0]
None
